In [1]:
import re
import pandas as pd
from os.path import join
from pandas import DataFrame
from pandas import HDFStore
from helper import page_view_stat
from helper import removeDataFrame

### 0. 准备工作

#### (1). 通用字段定义

In [2]:
typeDefines = {u'dev_id': str, u'stat_date': str, u'stat_time': str, u'post_id': int, 'type': int, 'stat_view': int, 'stat_click': int}
header = ['dev_id', 'stat_date', 'stat_time', 'post_id', 'type', 'stat_view', 'stat_click']
dtypes = [typeDefines[col] for col in header]
dbName = 'db.h5'

####  (2). 获得数据文件列表

In [3]:
fileList = !ls data
fileList = filter(lambda x: re.match(r'^\d{6}_\d$', x), fileList)
fileList = map(lambda x: join('data', x), fileList)

### 2. 整理train相关的page_view数据

In [4]:
dfName = 'page_view_train'

#### (1) 读取train清单

In [5]:
train = pd.read_csv('data/train.txt', sep='\t', names=[u'dev_id', u'post_id', u'praise', u'time'])
train = train.sort_values([u'dev_id', u'post_id', u'time', u'praise'])
train = train.groupby([u'dev_id', u'post_id'])['praise'].last().reset_index()
trainList = train[[u'dev_id', u'post_id']].drop_duplicates()

#### (2). 定义处理函数

In [6]:
def load_page_view_train(filename):
    """ """
    print '正在处理: %s' % filename, 
    # 按每次100万条方式处理, 数据文件
    for df in pd.read_csv(filename ,sep='\t', names=header, chunksize=1000000, converters=typeDefines):
        #  关联上train过滤调多余数据
        df = trainList.merge(df)
        # 导出到hdf文件
        df.to_hdf(dbName, dfName, append=True)
        print '.',
    print '(ok)'

#### (3). 生成page_view清单

In [7]:
%%time
removeDataFrame(dfName)
for filename in fileList:
    load_page_view_train(filename)

正在处理: data/000000_0 . . . . . . . . . . . . . . . . . . . . (ok)
正在处理: data/000001_0 . . . . . . . . . . . . . . . . . (ok)
正在处理: data/000002_0 . . . . . . . . . . . . . . . . (ok)
正在处理: data/000003_0 . . . . . . . . . . . . . . . (ok)
正在处理: data/000004_0 . . . . . . . . . . . . . . (ok)
正在处理: data/000005_0 . . . . . . . . . (ok)
正在处理: data/000006_0 . . . . . . (ok)
正在处理: data/000007_0 . . . (ok)
CPU times: user 4min 58s, sys: 8.34 s, total: 5min 6s
Wall time: 5min 12s


### 3. 关联trian和page_view数据生成train数据集

#### (1). 读取整理过的page_view清单

In [8]:
page_view_train = pd.read_hdf(dbName, dfName)
len(page_view_train)

909470

In [9]:
len(page_view_train[['dev_id','post_id']].drop_duplicates())

293288

#### (2). 统计汇总

In [10]:
df = page_view_stat(page_view_train)

In [11]:
len(df[['dev_id','post_id']].drop_duplicates())

293288

#### (3). 保存到训练集中

In [12]:
dfName = 'train'
removeDataFrame(dfName)
train.merge(df).to_hdf(dbName, dfName, append=True)